In [17]:
from crewai.flow.flow import Flow, listen, start, router, or_, and_
from pydantic import BaseModel, Field
from typing import List, Dict, Optional
from IPython.display import display
from random import randint

def log(msg: str) -> None:
    """Show message in Jupyter output (works inside CrewAI flow steps)."""
    display(msg)

In [18]:
# Define your state model
class UserPreferences(BaseModel):
    theme: str = "light"
    language: str = "English"
    __private_field: str = "This is a private field"

class AppState(BaseModel):
    user_name: str = ""
    preferences: UserPreferences = UserPreferences()
    items: List[str] = []
    processed: bool = False
    completion_percentage: float = 0.0

In [19]:
# Create a flow with typed state
class StructuredStateFlow(Flow[AppState]):
    @start()
    def initialize_data(self):
        log("Initializing flow data")
        # Set state values (type-checked)
        self.state.user_name = "Ali Khan"
        self.state.preferences.theme = "White"
        self.state.preferences.__private_field = " private field written again "
        return "Initialized data done"

    @listen(initialize_data)
    def process_data(self, previous_result):
        log(f"Processing data for {self.state.user_name}")

        # Modify state (with type checking)
        self.state.items.append("item1")
        self.state.items.append("item2")
        self.state.processed = True
        self.state.completion_percentage = randint(0, 100)
        return "Processed data done"

    @listen(process_data)
    def generate_summary(self, previous_result):
        
        summary = f"User {self.state.user_name} has {len(self.state.items)} items "
        summary += f"with {self.state.preferences.theme} theme. "
        summary += f"Private field: {self.state.preferences.__private_field}"
        summary += "Data is processed." if self.state.processed else "Data is not processed."
        summary += f" Completion: {self.state.completion_percentage}%"
        return summary
    
    @router(generate_summary)
    def router_function(self, summary):
        if self.state.completion_percentage < 30:
            log(f"Calling threshold_reached less then 30 {summary}")
            return "threshold_reachedless30"
        elif self.state.completion_percentage > 30 and self.state.completion_percentage < 40:
            log(f"Calling threshold_not_reached between 30 and 40 {summary}")
            return "threshold_not_reachedbetween30and40"
        else:
            log(f"Calling threshold_not_matched  {summary}")
            return "threshold_not_matched"
        
        
    @listen(or_("threshold_reachedless30", "threshold_not_reachedbetween30and40"))    
    def thread_reached_callback(self, previous_result):
        log(f"Threshold reached: {previous_result}")
        return "--Threshold reached"
    
    @listen("threshold_not_reached")
    def threshold_not_reached_callback(self, previous_result):
        log(f"Threshold not reached: {previous_result}")
        return "--Threshold not reached"



In [20]:
# Run the flow
flow = StructuredStateFlow()
result = await flow.kickoff_async()
print(f"Final result: {result}")
print(f"Final state: {flow.state}")

Flow started with ID: e5e96257-26ff-4e70-9b27-4ec7a74eebc0


Flow started with ID: e5e96257-26ff-4e70-9b27-4ec7a74eebc0


╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: StructuredStateFlow                                                                                      │
│  ID: e5e96257-26ff-4e70-9b27-4ec7a74eebc0                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

'Initializing flow data'

'Processing data for Ali Khan'

'Calling threshold_reached less then 30 User Ali Khan has 2 items with White theme. Private field:  private field written again Data is processed. Completion: 21%'

Output()

'Threshold reached: User Ali Khan has 2 items with White theme. Private field:  private field written again Data is processed. Completion: 21%'

╭──────────────────────────────────────────────── Flow Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Execution Completed                                                                                       │
│  Name: StructuredStateFlow                                                                                      │
│  ID: e5e96257-26ff-4e70-9b27-4ec7a74eebc0                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final result: --Threshold reached
Final state: id='e5e96257-26ff-4e70-9b27-4ec7a74eebc0' user_name='Ali Khan' preferences=UserPreferences(theme='White', language='English') items=['item1', 'item2'] processed=True completion_percentage=21
